# **Python for Finance -- Pandas -- Assignment**
---
<img src="http://www.doc.ic.ac.uk/~afd/images/logo_imperial_college_london.png" align = "left" width=200>
 <br><br><br><br>
 
- Copyright (c) Antoine Jacquier, 2022. All rights reserved

- Author: Antoine Jacquier <a.jacquier@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.7

# Problem 1: basics

Finish the exercises from the main notebook

# Problem 2: options data cleaning

The file ```AMZN_options.csv``` contains European options data on Amazon. 
Recall that an option is a financial derivative that pays at expiration time $T$ and strike $K>0$:

$$C(K,T)=(S_T-K)_+ \text{  for a Call option}$$

$$P(K,T)=(K-S_T)_+ \text{  for a Put option}$$

No-arbitrage theory tells us that Put-Call parity holds and takes the form

$$C(K,T)-P(K,T)=S(T)-K\cdot DF(T),$$

where $S(T)$ is the underlying price and $DF(T)$ is the discount factor at time $T$.

- Compute a new column ```mid_price``` corresponding to
$$\text{mid\_price}=\frac{\text{bid}+\text{ask}}{2}$$

- For each available ```expiration_date``` perform a linear regression using the ```mid_price``` only for options ```whose trade_volume>25``` (both calls and puts need to satisfy this condition):

$$C(K,T)-P(K,T)=a+b K,$$ 
where $a$ corresponds to $S(T)$ and $b$ corresponds to $DF(T)$.

- Plot $S(T)$ and  $DF(T)$ as a function of $T$

*Notes*: 

    + you may use `numpy.polyfit(x, y, deg=1)` to fit a linear regression and obtain the coefficients
    
    + maturities are quoted in years, so you may need to use the `datetime` package


In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import time

df_options_AMZ = pd.read_csv('AMZN_options.csv')
df_options_AMZ['mid_price'] = (df_options_AMZ['bid'] + df_options_AMZ['ask'])/2.
df_options_AMZ['expiration_date'] = [time.strptime(d, '%Y-%m-%d') for d in df_options_AMZ['expiration_date']]
df_options_AMZ.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration_date,option_type,mid_price
0,AMZN201106C01820000,2020-11-02 18:28:21,1820.0,1148.70,1139.35,1143.20,-62.51001,-5.160956,15.0,4.0,0.00001,True,REGULAR,USD,"(2020, 11, 6, 0, 0, 0, 4, 311, -1)",C,1141.275
1,AMZN201106C01830000,2020-11-02 18:28:21,1830.0,1146.25,1123.35,1127.75,-56.18994,-4.672994,1.0,2.0,0.00001,True,REGULAR,USD,"(2020, 11, 6, 0, 0, 0, 4, 311, -1)",C,1125.550
2,AMZN201106C01840000,2020-11-02 18:28:21,1840.0,1128.45,1116.55,1120.70,-62.75000,-5.267798,1.0,2.0,0.00001,True,REGULAR,USD,"(2020, 11, 6, 0, 0, 0, 4, 311, -1)",C,1118.625
3,AMZN201106C01850000,2020-11-02 18:28:21,1850.0,1126.00,1108.85,1112.60,-56.47998,-4.776401,1.0,3.0,0.00001,True,REGULAR,USD,"(2020, 11, 6, 0, 0, 0, 4, 311, -1)",C,1110.725
4,AMZN201106C01870000,2020-11-02 14:34:36,1870.0,1201.10,1086.10,1092.25,-17.23999,-1.415039,2.0,2.0,0.00001,True,REGULAR,USD,"(2020, 11, 6, 0, 0, 0, 4, 311, -1)",C,1089.175


In [17]:
df_calls = df_options_AMZ.loc[df_options_AMZ['option_type']=='C',:]
df_puts = df_options_AMZ.loc[df_options_AMZ['option_type']=='P',:]
df_calls_flat = df_calls.reset_index()
df_puts_flat = df_puts.reset_index()
# C = np.array(df_calls['mid_price'])
# P = np.array(df_options_AMZ.loc[df_options_AMZ['option_type']=='P',:]['mid_price'])

# Problem 3: implied volatility in the presence of interest rates
*(Continuation of the problem from Session 2)*

Using the forward prices $F(T)$ and Discount Factors $DF(T)$ obtained previously, calculate the implied volatility of each option using the  ```mid_price```. 
Recall that in the Black-Scholes model, the value of a European Call option on $(S_t)_{t\geq 0}$ is given at inception by

$$
C^{\mathrm{BS}}(S_0, K, T;\sigma) = S(T)\left(\mathcal{N}(d_{+}) - DF(T)K\mathcal{N}(d_{-})\right),
$$

with 

$$
d_{\pm} := \frac{\log\left(\frac{F(T)}{K}\right)}{\sigma\sqrt{T}} \pm\frac{\sigma\sqrt{T}}{2}
\qquad\text{and}\qquad
F(T)=\frac{S(T)}{DF(T)}.
$$

*Note: some mid prices might lead to arbitrage and the solution for implied volatility might not exist.*